# SQL Cheat Sheet
SUBQUERIES TEST

- Connecting to Database

In [1]:
reload_ext sql

In [2]:
%%sql
postgresql://localhost/countries

'Connected: @countries'

___

In [3]:
%%sql
SELECT * FROM economies
LIMIT 4

 * postgresql://localhost/countries
4 rows affected.


econ_id,code,year,income_group,gdp_percapita,gross_savings,inflation_rate,total_investment,unemployment_rate,exports,imports
1,AFG,2010,Low income,539.667,37.133,2.179,30.402,None,46.394,24.381
2,AFG,2015,Low income,615.091,21.466,-1.549,18.602,None,-49.11,-7.294
3,AGO,2010,Upper middle income,3599.27,23.534,14.48,14.433,None,-3.266,-21.076
4,AGO,2015,Upper middle income,3876.2,-0.425,10.287,9.552,None,6.721,-21.778


In [106]:
%%sql
SELECT DISTINCT region, count(*)  FROM countries
group by region

 * postgresql://localhost/countries
23 rows affected.


region,count
Southeast Asia,10
North America,4
Western Europe,9
Western Africa,16
Polynesia,5
Southern and Central Asia,14
Middle East,18
Eastern Asia,7
Southern Africa,5
Melanesia,5


In [109]:
%%sql
SELECT * FROM languages
LIMIT 4

 * postgresql://localhost/countries
4 rows affected.


lang_id,code,name,percent,official
1,AFG,Dari,50.0,True
2,AFG,Pashto,35.0,True
3,AFG,Turkic,11.0,False
4,AFG,Other,4.0,False


In [110]:
%%sql
SELECT * FROM populations
LIMIT 4

 * postgresql://localhost/countries
4 rows affected.


pop_id,country_code,year,fertility_rate,life_expectancy,size
20,ABW,2010,1.704,74.95354,101597.0
19,ABW,2015,1.647,75.573586,103889.0
2,AFG,2010,5.746,58.97083,27962208.0
1,AFG,2015,4.653,60.71717,32526562.0


___

## 1. Subquery Challenge 1: Mulitple Subqueries

- Determine for each of the continents in the countriest table, which county has the max inflation rate. in 2015

In [5]:
%%sql
SELECT code, inflation_rate AS inf 
FROM economies
WHERE year = 2015
LIMIT 2;

 * postgresql://localhost/countries
2 rows affected.


code,inf
AFG,-1.549
AGO,10.287


In [7]:
%%sql
SELECT continent, MAX(subquery.inf) AS max_inf
FROM countries, 
    (SELECT code, inflation_rate AS inf 
     FROM economies
     WHERE year = 2015) AS subquery
WHERE countries.code = subquery.code
GROUP BY continent

 * postgresql://localhost/countries
6 rows affected.


continent,max_inf
Africa,21.858
Asia,39.403
South America,121.738
North America,7.524
Europe,48.684
Oceania,9.784


- FINAL CODE

In [18]:
%%sql
SELECT name, second_query.continent, second_query.max_inf
FROM countries, economies,  
     (SELECT continent, MAX(subquery.inf) AS max_inf
      FROM countries, 
        (SELECT code, inflation_rate AS inf 
         FROM economies
         WHERE year = 2015) AS subquery
      WHERE countries.code = subquery.code
      GROUP BY continent) AS second_query
WHERE economies.inflation_rate = second_query.max_inf
AND countries.code = economies.code

 * postgresql://localhost/countries
6 rows affected.


name,continent,max_inf
Haiti,North America,7.524
Malawi,Africa,21.858
Nauru,Oceania,9.784
Ukraine,Europe,48.684
Venezuela,South America,121.738
Yemen,Asia,39.403


#### USING JOINS

In [20]:
%%sql
SELECT name, continent, inflation_rate
FROM countries
  INNER JOIN economies
    USING (code)
WHERE year = 2015
AND inflation_rate IN 
    (SELECT MAX(inflation_rate) AS max_inf
        FROM 
            (SELECT name, continent, inflation_rate
             FROM countries
             INNER JOIN economies
             USING (code)
             WHERE year = 2015) AS subquery
             GROUP BY continent)

 * postgresql://localhost/countries
6 rows affected.


name,continent,inflation_rate
Haiti,North America,7.524
Malawi,Africa,21.858
Nauru,Oceania,9.784
Ukraine,Europe,48.684
Venezuela,South America,121.738
Yemen,Asia,39.403


___

## 2.Subquery Challenge 2

In [91]:
%%sql
SELECT * FROM countries
LIMIT 1

 * postgresql://localhost/countries
1 rows affected.


code,name,continent,region,surface_area,indep_year,local_name,gov_form,capital,cap_long,cap_lat
AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919,Afganistan/Afqanestan,Islamic Emirate,Kabul,69.1761,34.5228


In [94]:
%%sql
SELECT * FROM economies
WHERE year = 2015
LIMIT 1

 * postgresql://localhost/countries
1 rows affected.


econ_id,code,year,income_group,gdp_percapita,gross_savings,inflation_rate,total_investment,unemployment_rate,exports,imports
2,AFG,2015,Low income,615.091,21.466,-1.549,18.602,None,-49.11,-7.294


____

- Use a subquery to get 2015 economic data (inflation & unemployment) for countries that do not have
    - gov_form of 'Constitutional Monarchy' or 
    - 'Republic' in their 'gov_form'

In [70]:
%%sql
SELECT name, gov_form, subquery.inflation_rate, subquery.unemployment_rate
FROM countries,
    (SELECT code, inflation_rate, unemployment_rate 
    FROM economies
    WHERE year = 2015) AS subquery
WHERE gov_form <> 'Constitutional Monarchy'
AND gov_form NOT LIKE '%Republic%'
AND countries.code = subquery.code
ORDER BY subquery.inflation_rate

 * postgresql://localhost/countries
21 rows affected.


name,gov_form,inflation_rate,unemployment_rate
Afghanistan,Islamic Emirate,-1.549,None
Switzerland,Federation,-1.14,3.178
Puerto Rico,Commonwealth of the US,-0.751,12.0
Brunei,Monarchy (Sultanate),-0.423,6.9
Tonga,Monarchy,-0.283,None
Oman,Monarchy (Sultanate),0.065,None
Belgium,"Constitutional Monarchy, Federation",0.62,8.492
Canada,"Constitutional Monarchy, Federation",1.132,6.9
Australia,"Constitutional Monarchy, Federation",1.461,6.058
Qatar,Monarchy,1.814,None


- DataCamp's Answer. This is wrong because it selects countries tha are not in the countries table. 
- My answer says give economic data ONLY on the list of countries from the countries table that have gov_form <> Constitutional Monarchy' and not like '%Republic%'
- Theirs says give economic data on countries that are not in the list of countries (from the countries table) that have gov_form = 'Constitutional Monarchy' or like '%Republic%'
- If both countries and economies tables had the exact amoutn of countries it would be fine, but the 'economies table' has countries that are not in the 'countries table' and their solution includes said countries

In [98]:
%%sql
SELECT economies.code, inflation_rate, unemployment_rate
FROM economies
WHERE year = 2015
AND economies.code NOT IN
   (SELECT code
   FROM countries
   WHERE (gov_form  = 'Constitutional Monarchy' OR gov_form LIKE '%Republic%'))
ORDER BY inflation_rate;

 * postgresql://localhost/countries
26 rows affected.


code,inflation_rate,unemployment_rate
AFG,-1.549,None
CHE,-1.14,3.178
PRI,-0.751,12.0
ROU,-0.596,6.812
BRN,-0.423,6.9
TON,-0.283,None
OMN,0.065,None
TLS,0.553,None
BEL,0.62,8.492
CAN,1.132,6.9


___

## 3. Subquery Challenge 3

- 2015 economic data (investment & imports) on countries in Central America with and official language

In [108]:
%%sql
SELECT DISTINCT c.name, e.total_investment,e.imports
FROM countries AS c
    LEFT JOIN economies AS e
        ON (c.code = e.code)
        AND c.code IN 
        ( SELECT l.code
          FROM languages AS l
          WHERE l.official = True) 
WHERE c.region = 'Central America' AND year = 2015
ORDER BY c.name;

 * postgresql://localhost/countries
7 rows affected.


name,total_investment,imports
Belize,22.014,6.743
Costa Rica,20.218,4.629
El Salvador,13.983,8.193
Guatemala,13.433,15.124
Honduras,24.633,9.353
Nicaragua,31.862,11.665
Panama,46.557,5.898


___

## 4. Subquery Challenge 4

- Average fertility rate in 2015

In [115]:
%%sql
SELECT c.region, c.continent, AVG(fertility_rate) AS avg_fert_rate
FROM countries AS c
    INNER JOIN populations AS p
        ON c.code = p.country_code
        WHERE year = 2015
GROUP BY c.region, c.continent
ORDER BY avg_fert_rate;

 * postgresql://localhost/countries
23 rows affected.


region,continent,avg_fert_rate
Southern Europe,Europe,1.4261000037193299
Eastern Europe,Europe,1.490888900227017
Baltic Countries,Europe,1.603333314259847
Eastern Asia,Asia,1.6207143068313599
Western Europe,Europe,1.6325000077486038
North America,North America,1.7657500207424164
British Islands,Europe,1.875
Nordic Countries,Europe,1.8933333555857341
Australia and New Zealand,Oceania,1.9114999771118164
Caribbean,North America,1.9505714348384313


___

## 5. Subquery Challenge 5

- Top 10 capital cities in Europe and the Americas in terms of a calculated percentage using city_proper_pop and metroarea_pop in cities.
- percentage of metro area population composed of city proper population for each city 

In [132]:
%%sql
SELECT name, country_code, city_proper_pop, metroarea_pop, city_proper_pop / metroarea_pop * 100 AS city_perc
FROM cities
WHERE name IN
    (SELECT capital
     FROM countries
     WHERE continent = 'Europe' OR countries.continent LIKE '%America')
AND metroarea_pop IS NOT NULL
ORDER BY city_perc DESC
LIMIT 10;


 * postgresql://localhost/countries
10 rows affected.


name,country_code,city_proper_pop,metroarea_pop,city_perc
Lima,PER,8852000.0,10750000.0,82.34418630599976
Bogota,COL,7878783.0,9800000.0,80.3957462310791
Moscow,RUS,12197596.0,16170000.0,75.43349266052246
Vienna,AUT,1863881.0,2600000.0,71.6877281665802
Montevideo,URY,1305082.0,1947604.0,67.00961589813232
Caracas,VEN,1943901.0,2923959.0,66.48181676864624
Rome,ITA,2877215.0,4353775.0,66.0855233669281
Brasilia,BRA,2556149.0,3919864.0,65.2101457118988
London,GBR,8673713.0,13879757.0,62.491822242736816
Budapest,HUN,1759407.0,2927944.0,60.09018421173096
